In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
from evallm.utils.bootstrap import boostrap_mean

from evallm.experiments.evaluate_tokenization_regexp_non_icl import *
import re
import pandas as pd

from evallm.experiments.models_display import model_by_display_key
from evallm.experiments.main_tables import models_considered

In [3]:
def for_models(*model_keys, amount=100):
    return

In [4]:
all_model_results = {
    model_key: evaluate_model_regexp_matching_multiple(
        model_by_display_key[model_key], count=100
    )
    for model_key in model_by_display_key
}

In [5]:
summary(*all_model_results["gpt-4o"])

Percent positive: 53%
Percent correct : 100%


In [6]:
all_model_nr = {k: (p == -1).mean() for k, (p, t) in all_model_results.items()}
all_model_accs = {
    k: (p == t).mean() / (p != -1).mean() if (p != -1).any() else 0
    for k, (p, t) in all_model_results.items()
}

In [7]:
all_model_accs_keys = list(all_model_accs)
all_model_accs_values = [
    dict(acc=all_model_accs[k], non_response=all_model_nr[k])
    for k in all_model_accs_keys
]
table = pd.DataFrame(all_model_accs_values, index=all_model_accs_keys)
table = table.loc[sorted(table.index, key=models_considered.index)]
table

,acc,non_response
llama3-8B,0.660000,0.00
llama3-70B,0.797297,0.26
llama3.1-8B-Instruct,0.830000,0.00
llama3.1-8B,0.890000,0.00
llama3.1-70B,0.859375,0.36
qwen-2.5-7B,0.620000,0.00
qwen-2.5-32B,1.000000,0.00
mistral-nemo-minitron-8B,0.000000,1.00
mistral-nemo-base-12B,1.000000,0.99
mistral-nemo-instruct-12B,0.990000,0.00


In [8]:
non_response_threshold = 0.98
table[table.non_response > non_response_threshold]

,acc,non_response
mistral-nemo-minitron-8B,0.0,1.00
mistral-nemo-base-12B,1.0,0.99
falcon-7b,0.0,1.00
deepseek-coder-33b-instruct,0.0,1.00


In [9]:
for k in table[table.non_response < non_response_threshold].index:
    render = f"{k:30s} {table.acc[k]:4.0%}"
    if table.non_response[k] > 0:
        render += f" [Non-response: {table.non_response[k]:.0%}]"
    print(render)

llama3-8B                       66%
llama3-70B                      80% [Non-response: 26%]
llama3.1-8B-Instruct            83%
llama3.1-8B                     89%
llama3.1-70B                    86% [Non-response: 36%]
qwen-2.5-7B                     62%
qwen-2.5-32B                   100%
mistral-nemo-instruct-12B       99%
gemma-7b                        67% [Non-response: 15%]
starcoder2-15b                  62%
codestral-22B                   95%
qwen-2.5-coder-7B              100%
qwen-2.5-coder-instruct-7B      56%
qwen-2.5-coder-instruct-32B     96%
gpt-3.5-instruct                67%
gpt-3.5-chat                    82%
gpt-4o-mini                    100%
gpt-4o                         100%
claude-3.5                     100%
o3-mini                        100%
